In [ ]:
# camera settings
hfov = 67.0
wr = 1280
avg_shoulder_width = 350.0
avg_torso_height = 460.0

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import shutil
from scipy.interpolate import RectBivariateSpline
import time
from IPython.display import clear_output
import queue
import math

from dataclasses import dataclass

@dataclass
class Point:
    x: int = 0
    y: int = 0
    z: int = 0

    def distance(self, other):
        return np.sqrt((self.x - other.x) ** 2 + (self.y - other.y) ** 2 + (self.z - other.z) ** 2)
    
    def middle(self, other):
        return Point((self.x + other.x) / 2, (self.y + other.y) / 2, (self.z + other.z) / 2)


def distance_to_camera(hfov, wr, measured_width, pixel_width):
    return (measured_width * wr) / (pixel_width * 2 * math.tan(math.radians(hfov / 2)))


#To Clear the model cache
# shutil.rmtree(torch.hub.get_dir(), ignore_errors=True)

#Initializing the body landmarks detection module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

prev = None
cap = cv2.VideoCapture(0)
time.sleep(0.5)

while cap.isOpened():
    ret, img = cap.read()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img)

    if results.pose_landmarks is not None:
        # Draw Landmarks
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

        left_shoulder_point = Point(
            int(left_shoulder.x * img.shape[1]), 
            int(left_shoulder.y * img.shape[0]), 
            int(left_shoulder.z * img.shape[0]))
        right_shoulder_point = Point(
            int(right_shoulder.x * img.shape[1]), 
            int(right_shoulder.y * img.shape[0]), 
            int(right_shoulder.z * img.shape[0]))
        
        left_hip_point = Point(
            int(left_hip.x * img.shape[1]), 
            int(left_hip.y * img.shape[0]), 
            int(left_hip.z * img.shape[0]))
        right_hip_point = Point(
            int(right_hip.x * img.shape[1]), 
            int(right_hip.y * img.shape[0]), 
            int(right_hip.z * img.shape[0]))
        
        middle_shoulder_point = left_shoulder_point.middle(right_shoulder_point)
        middle_hip_point = left_hip_point.middle(right_hip_point)
        torso_height_pixel = middle_shoulder_point.distance(middle_hip_point)
        torso_height = distance_to_camera(hfov, wr, avg_torso_height, torso_height_pixel)

        shoulder_pixel_width = left_shoulder_point.distance(right_shoulder_point)
        shoulder_width = distance_to_camera(hfov, wr, avg_shoulder_width, shoulder_pixel_width)

        print(f'{shoulder_width:.0f} mm')
        print(f'{torso_height:.0f} mm')
        mean_distance = (shoulder_width + torso_height) / 2
        print(f'{max(shoulder_width, torso_height):.0f} mm')

    clear_output(wait=True)
    fig, ax = plt.subplots()
    ax.imshow(img)
    #scatter red dot at middle_shoulder_point
    ax.scatter(middle_shoulder_point.x, middle_shoulder_point.y, c='r', s=10)
    #scatter red dot at middle_hip_point
    ax.scatter(middle_hip_point.x, middle_hip_point.y, c='r', s=10)
    plt.axis('off')
    plt.show()

    # Frame rate
    plt.pause(0.15)
    
